In [1]:
%xmode verbose
import torch
import detectron2
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
import layoutparser as lp

from pdf2image import convert_from_path
from PIL import Image

#For Detectron2
import torch
import detectron2
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
import layoutparser as lp

import pytesseract
from pytesseract import Output
import matplotlib.pyplot as plt

from PIL import Image
import numpy as np
import cv2

import os

os.chdir('..')

path= ''
prediction_score_threshold = 0.7
class_labels = ['text', 'title', 'list', 'table', 'figure']

# Set up Detectron2 config
cfg = get_cfg()
#The following line can be used instead of the one after it, however it causes warnings. (Find out why?)
#cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set a threshold for predictions
cfg.MODEL.WEIGHTS = path+ "model_final.pth" 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = prediction_score_threshold
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5

cfg.MODEL.DEVICE = "cpu"



predictor = DefaultPredictor(cfg)

Exception reporting mode: Verbose


In [2]:
def detectron_boxes(images_folder, output_folder):
    """
    Extracts the boxes and classes from images
    
    Args:
    images_folder (String): Path to the images folder
    output_folder (String): The location where all extracted information will be saved as txt files
    
    Returns:
    For each image it creates a text file containing the relevant information
    """
    
    path = ''
    prediction_score_threshold = 0.7
    class_labels = ['text', 'title', 'list', 'table', 'figure']

    # Set up Detectron2 config
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set a threshold for predictions
    cfg.MODEL.WEIGHTS = path + "model_final.pth"  # e.g., PubLayNet pre-trained model
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = prediction_score_threshold
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5

    cfg.MODEL.DEVICE = "cpu"

    # Initialize an empty list to store the content of each detected object
    detected_texts = []

    predictor = DefaultPredictor(cfg)
    
    # Ensure output folder exists
    if not os.path.exists(output_folder):
        print(f"Output folder does not exist, creating: {output_folder}")
        os.makedirs(output_folder)

    for image_name in os.listdir(images_folder):
        image_path = os.path.join(images_folder, image_name)
        current_image = cv2.imread(image_path)
        img = np.array(current_image)

        # Perform page object detection
        outputs = predictor(img)

        # Debug outputs
        instances = outputs["instances"].to("cpu")
        pred_boxes = instances.pred_boxes
        scores = instances.scores
        pred_classes = instances.pred_classes
        
        #print(f"Image: {image_name}")
        #print(f"Scores: {scores}")
        #print(f"Bounding Boxes: {pred_boxes}")

        # Only process if there are valid predictions
        if len(scores) == 0:
            print("No detections found in this image.")
            continue
        
        # Saving results
        results_path = os.path.join(output_folder, f"{os.path.splitext(image_name)[0]}.txt")
        #print(f"Saving results to: {results_path}")
        
        try:
            with open(results_path, "w") as f:
                f.write(f"Image: {image_name}\n")
                f.write(f"{'Class':<10}{'Score':<10}{'Bounding Box':<30}\n")
                f.write("-" * 50 + "\n")
                for score, box, cls in zip(scores, pred_boxes, pred_classes):
                    if score >= prediction_score_threshold:
                        #print(f"Writing detection: Score: {score}, Class: {cls}, Box: {box}")
                        label = class_labels[cls]
                        box_str = f"[{box[0]:.2f}, {box[1]:.2f}, {box[2]:.2f}, {box[3]:.2f}]"
                        f.write(f"{label:<10}{score:<10.2f}{box_str:<30}\n")
        except Exception as e:
            print(f"Failed to write to file {results_path}: {e}")
    
    print(f"Done")


In [3]:
%xmode verbose

detectron_boxes('./val_images','./output')

Exception reporting mode: Verbose


The checkpoint state_dict contains keys that are not used by the model:
  roi_heads.mask_head.mask_fcn1.{bias, weight}
  roi_heads.mask_head.mask_fcn2.{bias, weight}
  roi_heads.mask_head.mask_fcn3.{bias, weight}
  roi_heads.mask_head.mask_fcn4.{bias, weight}
  roi_heads.mask_head.deconv.{bias, weight}
  roi_heads.mask_head.predictor.{bias, weight}
/usr/local/lib/python3.9/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Done


In [7]:
import os

# Get the current working directory
current_path = './tf'
# List all items in the current directory
items = os.listdir(current_path)

# Print all items
print(f"Items in the current folder ({current_path}):")
for item in items:
    print(item)



Items in the current folder (./tf):
.ipynb_checkpoints
bashrc
detectronEval.ipynb
docker-compose.yaml
gpu.packages.txt
gpu.requirements.txt
jupyter.Dockerfile
jupyter.readme.md
jupyter.requirements.txt
output
PDF_docker_showcase.ipynb
setup.cuda.sh
setup.jupyter.sh
setup.packages.sh
setup.python.sh
setup.sources.sh
test
test.import_cpu.sh
val_images
